In [ ]:
from common import *

In [ ]:
client = start_cluster(5, 12)
client_ip_and_port(client)

# SWE output comparison between iSnobal and Snow-17

## CBRFC

In [ ]:
CBRFC_CSV = pd.read_csv(
    CBRFC_DIR / 'SNOW17ALEC2_2018to2021.csv',
    parse_dates=True,
    index_col=0,
    header=0,
    names=['Lower', 'Middle', 'Upper', 'Average'],
    dtype={'Lower': np.float64, 'Middle': np.float64, 'Upper': np.float64},
)
CBRFC_CSV.Lower *= INCH_TO_MM
CBRFC_CSV.Middle *= INCH_TO_MM
CBRFC_CSV.Upper *= INCH_TO_MM

## iSnobal 

In [ ]:
wy_swe = xr.open_mfdataset(
    (SNOBAL_DIR / 'GMD/wy20*' / 'erw/*/snow.nc').as_posix(),
    parallel=True,
)

wy_swe.coords['mask'] = (('y', 'x'), cbrfc_zones())

In [ ]:
hlf = wy_swe.where(wy_swe.mask == ALEC2HLF).specific_mass.mean(("x", "y")).compute()
hmf = wy_swe.where(wy_swe.mask == ALEC2HMF).specific_mass.mean(("x", "y")).compute()
huf = wy_swe.where(wy_swe.mask == ALEC2HUF).specific_mass.mean(("x", "y")).compute()

In [ ]:
client.shutdown()

## Compare CBRFC to iSnobal

In [ ]:
plt.figure(dpi=300, figsize=(9,3))
ax = plt.gca()

linestyle=dict(alpha=0.7, lw=1)

ax.plot([], [], ' ', label=r'Lower', lw=0)
ax.plot([], [], ' ', label=r'$\ $', lw=0)
hlf.plot(ax=ax, label='HRRR-iSnobal', color='gold', **linestyle)
CBRFC_CSV.Lower.plot(ax=ax, color='orange', label='SNOW-17', ls='--', **linestyle)
ax.plot([], [], ' ', label=r'Middle')
ax.plot([], [], ' ', label=r'$\ $', lw=0)
hmf.plot(ax=ax, label='HRRR-iSnobal', color='teal', **linestyle)
CBRFC_CSV.Middle.plot(ax=ax, color='cadetblue', label='SNOW-17', ls='--', **linestyle)
ax.plot([], [], ' ', label=r'Upper')
ax.plot([], [], ' ', label=r'$\ $', lw=0)
huf.plot(ax=ax, label='HRRR-iSnobal', color='indigo', **linestyle)
CBRFC_CSV.Upper.plot(ax=ax, color='mediumorchid', label='SNOW-17', ls='--', **linestyle)

locator = mdates.AutoDateLocator(maxticks=17)

ax.xaxis.set_major_locator(locator)
ax.xaxis.set_minor_locator(mdates.MonthLocator())

formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_formatter(formatter)

ax.set_xlim(left=CBRFC_CSV.Upper.index.min(), right=huf.time.max().values)

index = 0
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=0, horizontalalignment='center')
    if index in [1, 5, 9, 13]:
        label.set(fontstyle='italic', fontweight=600)
    index += 1

         
plt.ylabel('SWE (mm)')
plt.xlabel('')
         
legend = plt.legend(
    bbox_to_anchor=(0.475, 1.22), loc='upper center', 
    frameon=False, ncol=6, columnspacing=1.1, borderaxespad=0
)
         
for text in legend.get_texts():
    if text.get_text() in ['Upper', 'Middle', 'Lower']:
        text.set_fontstyle('italic')

In [ ]:
stats = np.zeros([9,4])
start_year = 2017

for y_index in range(0, stats.shape[1]):
    year = start_year + y_index
    time=slice(f"{year}-10-01", f"{year + 1}-09-30")
    
    stats[0, y_index] = huf.sel(time=time).sum()
    stats[1, y_index] = CBRFC_CSV.Upper[time].sum()
    stats[2, y_index] = stats[0, y_index] / stats[1, y_index]

    stats[3, y_index] = hmf.sel(time=time).sum()
    stats[4, y_index] = CBRFC_CSV.Middle[time].sum()
    stats[5, y_index] = stats[3, y_index] / stats[4, y_index]
    
    stats[6, y_index] = hlf.sel(time=time).sum()
    stats[7, y_index] = CBRFC_CSV.Lower[time].sum()
    stats[8, y_index] = stats[6, y_index] / stats[7, y_index]

In [ ]:
results = pd.DataFrame(
    stats, 
    columns=range(2018, 2022), 
    index=pd.MultiIndex.from_product([
        ['Upper', 'Middle', 'Lower'], 
        ['iSnobal', 'Snow-17', 'iSnobal/Snow-17']
    ])
)

headers = {
    'selector': 'th:not(.index_name)',
    'props': 'font-size: 16pt;'
}

ratios = pd.IndexSlice[:, 'iSnobal/Snow-17', :]

results.style.set_table_styles(
    [headers]
).set_properties(
    **{'font-size': '12pt'}
).format('{:.0f} mm').format('{:.0%}', subset=ratios)